In [1]:
# Code #1: merges OEP data, at county FIPs level, with KFF projected ACA increases, at congressional district level, using MSDC_geocorr mapping file.
# Weighted, county-level projected premium increase for benchmark silver plan for 60yo @ 400% FPL is saved in "final_county_shock" df
# Final_county_shock is merged with various tabs in 2025 OEP PUF (i.e. the tabs with data columns used to create the Access Disruption Risk Index i.e. Target Variable Y)
# Access Disruption Risk Index is generated and appended in master_df.xlsx
# master_df.xlsx is the key output that is then  used in future code.

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
OEP_2025_demographic = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(5) by Demographics')
OEP_2025_metal = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(6) by Metal Level')
OEP_2025_FPL = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(7) by FPL')
OEP_2025_FA = pd.read_excel('data/2025 OEP County-Level Public Use File.xlsx', sheet_name='(4) by Premium and FA')

MSDC_geocorr = pd.read_excel('data/geocorr2022_2534100191_HL.xlsx', sheet_name='Sheet1')

KFF = pd.read_excel('data/data-yiOuh_KFF_proj_ACA_increase.xlsx')

State_FIPS = pd.read_excel('data/StateFIPs.xlsx')

In [3]:
OEP_2025_demographic.head(2)
OEP_2025_FA.head(2)
State_FIPS.head(2)

,State (FIPS),State,State_abbv
0,1,Alabama,AL
1,2,Alaska,AK


In [4]:
## Create a new file of population-weighted average of ACA projected premium increase (from KFF) at the county level (e.g. some counties fall between 2 CD lines).
## This is being done since KFF data was originally only at CD level.

### Data Cleaning & Standardization (convert CD #'s from int to string)

MSDC_geocorr['state_abbv'] = MSDC_geocorr['state_abbv'].astype(str).str.strip()
MSDC_geocorr['cd119_code'] = MSDC_geocorr['cd119_code'].astype(int).astype(str)

KFF['state_abbv'] = KFF['state_abbv'].astype(str).str.strip()
KFF['cd119'] = KFF['cd119'].astype(int).astype(str)

#Keep all county pieces from Geocorr and attach KFF data to them (LEFT JOIN).
merged_df = pd.merge(
    MSDC_geocorr,
    KFF,
    left_on=['state_abbv', 'cd119_code'],
    right_on=['state_abbv', 'cd119'],
    how='left'
)

#Calculate Weighted Averages; select columns
shock_columns = [
    'benchmark_60_401_diff', 'benchmark_60_401_pct',
    'benchmark_40_401_diff', 'benchmark_40_401_pct',
    'benchmark_60_701_diff', 'benchmark_60_701_pct',
    'benchmark_40_701_diff', 'benchmark_40_701_pct'
]

#Create weighted columns for every metric
for col in shock_columns:
    weighted_col_name = f'weighted_{col}'
    merged_df[weighted_col_name] = merged_df[col] * merged_df['county-to-cd119_ allocation_factor']

#Aggregate to county level
agg_dict = {
    'state_abbv': 'first',
    'county-to-cd119_ allocation_factor': 'sum'
}
for col in shock_columns:
    agg_dict[f'weighted_{col}'] = 'sum'

final_county_shock = merged_df.groupby('cnty_FIPS_code').agg(agg_dict).reset_index()

#Rename columns to remove 'weighted_' prefix for final dataset cleanliness
final_rename_dict = {f'weighted_{col}': col for col in shock_columns}
final_county_shock.rename(columns=final_rename_dict, inplace=True)

#OUTPUT & QC
#Check for counties where allocation factor != 1.0 (indicating data gaps)
bad_matches = final_county_shock[
    (final_county_shock['county-to-cd119_ allocation_factor'] < 0.98) | 
    (final_county_shock['county-to-cd119_ allocation_factor'] > 1.02)
]

print("Merged Data Preview:")
print(final_county_shock.head())

if not bad_matches.empty:
    print(f"\nWARNING: {len(bad_matches)} counties have incomplete mappings (Factor != 1.0)")
    print(bad_matches.head())

#final_county_shock is a county FIPS level data file containing weighted averages for changes in premium (in both $ and %) for 60yo/401% FPL, 40yo/401% FPL, 60yo/701% FPL, 40yo/701%FPL
#final_county_shock.to_excel("final_county_shock.xlsx")

Merged Data Preview:
   cnty_FIPS_code state_abbv  county-to-cd119_ allocation_factor  \
0            1001         AL                                 1.0   
1            1003         AL                                 1.0   
2            1005         AL                                 1.0   
3            1007         AL                                 1.0   
4            1009         AL                                 1.0   

   benchmark_60_401_diff  benchmark_60_401_pct  benchmark_40_401_diff  \
0                 935.71                210.50                 205.41   
1                 901.52                202.80                 189.31   
2                 918.80                206.69                 197.45   
3                 935.71                210.50                 205.41   
4                 913.02                205.39                 194.73   

   benchmark_40_401_pct  benchmark_60_701_diff  benchmark_60_701_pct  \
0                 46.21                 603.14             

In [6]:
## Definition for access disruption risk factor:

![image.png](AccessDisruptionRiskIndex.png)

In [6]:
##Create a single df with data columns needed to create the access disruption risk factor
#Pop > 400% FPL = OEP_2025_FPL
#Pop 55yo+ = OEP_2025_demographic
#Delta Premium ($) = final_county_shock
#Pop CSR = OEP_2025_FA

#Ensure 'County FIPS Code' is a 5-digit string in ALL dataframes
dfs_to_clean=[final_county_shock, OEP_2025_demographic, OEP_2025_FPL, OEP_2025_FA]

for df in dfs_to_clean:
    df.columns = df.columns.str.strip()
    #Ensure it exists and is string type, padded with zeros if read as int (e.g. 1001 -> '01001')
    if 'County FIPS Code' in df.columns:
         df['County FIPS Code'] = df['County FIPS Code'].astype(str).str.zfill(5)
    #Handle the mapped column name in final_county_shock if it's different (e.g. 'cnty_FIPS_code')
    if 'cnty_FIPS_code' in df.columns:
         df.rename(columns={'cnty_FIPS_code': 'County FIPS Code'}, inplace=True)
         df['County FIPS Code'] = df['County FIPS Code'].astype(str).str.zfill(5)

# Clean State_FIPS and convert to string type
if 'State (FIPS)' in State_FIPS.columns:
    State_FIPS['State (FIPS)'] = State_FIPS['State (FIPS)'].astype(str).str.zfill(2)

#Rename Key Columns for Clarity/Safety before Merging
#This prevents column name collisions (e.g., having 3 "Number of Consumers..." columns)
OEP_2025_demographic = OEP_2025_demographic.rename(columns={
    'Age 55-64': 'count_age_55_64',
    'Age ≥ 65': 'count_age_65_plus',
    'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_master'
})

OEP_2025_FPL = OEP_2025_FPL.rename(columns={
    '≥100% to ≤150% of FPL': 'count_gt_100_150_fpl',
    '>150% to ≤200% of FPL': 'count_gt_150_200_fpl',
    '>400% of FPL': 'count_gt_400_fpl'
    # Rename this to drop it later or use for QC, not used for the calculation
    #'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_fpl_file'
})

OEP_2025_FA = OEP_2025_FA.rename(columns={
    'Consumers with CSRs': 'count_csr',
    # Rename this to drop it later or use for QC, not used for the calculation
    #'Number of Consumers with an Exchange Plan Selection': 'total_enrollees_fa_file'
})

#---------------------------------------------------------
#MERGE THE DATASETS
#---------------------------------------------------------
#Start with the Demographic file as the base (it contains the Age info and Master Total)
master_df = OEP_2025_demographic[['County FIPS Code', 'total_enrollees_master', 'count_age_55_64', 'count_age_65_plus']].copy()

# Create a merge key from the first 2 digits of the County FIPS
master_df['State_FIPS_Key'] = master_df['County FIPS Code'].str[:2]

#Merge 0: Add State_FIPS code
master_df = pd.merge(
    master_df,
    State_FIPS[['State (FIPS)', 'State']],
    left_on='State_FIPS_Key',
    right_on='State (FIPS)',
    how='left'
)

#Reorder columns to put State next to FIPS and drop helper columns
cols = ['County FIPS Code', 'State'] + [c for c in master_df.columns if c not in ['County FIPS Code', 'State', 'State_FIPS_Key', 'State (FIPS)']]
master_df = master_df[cols]

#Merge 1: Add FPL Data (100-150% FPL, 150-250% FPL, >400% FPL)
master_df = pd.merge(
    master_df,
    OEP_2025_FPL[['County FIPS Code', 'count_gt_400_fpl','count_gt_100_150_fpl','count_gt_150_200_fpl']],
    on='County FIPS Code',
    how='left'
)

#Merge 2: Add Financial Assistance Data (CSRs)
#master_df = pd.merge(
#    master_df,
#    OEP_2025_FA[['County FIPS Code', 'count_csr']],
#    on='County FIPS Code',
#    how='left'
#)

#Merge 3: Add the KFF Financial Shock Multiplier (add the 'benchmark_60_401_diff' column from weighted average calculation)
master_df = pd.merge(
    master_df,
    final_county_shock[['County FIPS Code', 'benchmark_60_401_diff']],
    on='County FIPS Code',
    how='inner' # Inner join ensures we only score counties where we have both Enrollment AND Cost data
)

#---------------------------------------------------------
#HANDLE MISSING VALUES (Data Cleaning)
#---------------------------------------------------------
#CMS PUFs often use "*" or empty strings for suppressed low values (e.g. <10 people); convert these to 0 or numeric types.
cols_to_numeric = ['total_enrollees_master', 'count_age_55_64', 'count_age_65_plus',
                   'count_gt_400_fpl', 'count_gt_100_150_fpl', 'count_gt_150_200_fpl', 'benchmark_60_401_diff']

for col in cols_to_numeric:
    #Coerce errors='coerce' turns non-numeric (like '*') into NaN, then fill with 0
    master_df[col] = pd.to_numeric(master_df[col], errors='coerce').fillna(0)

#---------------------------------------------------------
#CALCULATE RISK SCORE COMPONENTS
#---------------------------------------------------------

#1. Create the "Age Vulnerability Factor" (55+ Population / Total)
master_df['count_55_plus'] = master_df['count_age_55_64'] + master_df['count_age_65_plus']
master_df['factor_age_55_plus'] = master_df['count_55_plus'] / master_df['total_enrollees_master']      # % of 55+ enrollees in a county

#2. Calculate V1: LOW INCOME GROUP (% enrollees in 100-150% FPL)
master_df['V1_lowincome'] = master_df['count_gt_100_150_fpl'] / master_df['total_enrollees_master']

#3. Calculate V2: DOWNGRADE/OOP RISK GROUP (% enrollees in 150-250% FPL)
master_df['V2_ooprisk'] = master_df['count_gt_150_200_fpl'] / master_df['total_enrollees_master']

#4. Calculate V3: SUBSIDY CLIFF RISK GROUP (% enrollees in 400% FPL) * (% enrollees age 55yo+)
master_df['V3_Subsidycliff'] = (master_df['count_gt_400_fpl'] / master_df['total_enrollees_master']) * master_df['factor_age_55_plus'] # % 400% FPL * % 55yo+

#3. Calculate Risk A: LOW INCOME/OOP RISK
#Formula: (1.5*V1 + V2) * (Fixed $7,100 OOP Shock)
FIXED_OOP_SHOCK = 7100
master_df['raw_risk_lowincome'] = (
    (1.5*master_df['V1_lowincome'] + master_df['V2_ooprisk']) * FIXED_OOP_SHOCK
)

#X. Calculate Risk B: SUBSIDY CLIFF RISK
#Formula: (% Pop >400% FPL) * (Age 55+ Factor) * ($ Shock for 60yo)
master_df['raw_risk_cliff'] = (
    (master_df['V3_Subsidycliff']) * master_df['benchmark_60_401_diff']
)

# Do a log transform on the raw risk score to fix the skew in data where we have a few outlier counties at the top of the V3 risk and bring outliers closer to make distribution more normal
master_df['log_raw_risk_cliff'] = np.log1p(master_df['raw_risk_cliff'])

# ---------------------------------------------------------
# 6. FINAL SCORE GENERATION (Weighted and Normalized)
# ---------------------------------------------------------

# --- CREATE CONFIGURABLE WEIGHTS ---
# Adjust these variables to change the importance of each risk. They should sum to 1.0 (100%).
WEIGHT_CLIFF_RISK = 0.5      # Weight for Disenrollment/Flight Risk
WEIGHT_LOWINCOME_RISK = 1-WEIGHT_CLIFF_RISK  # Weight for Adherence/Downgrade Risk

# 1. Normalize both components to a 0-100 scale first
# This ensures "Max Risk" in one category is mathematically equal to "Max Risk" in the other
min_cliff = master_df['log_raw_risk_cliff'].min()
max_cliff = master_df['log_raw_risk_cliff'].max()
master_df['norm_risk_cliff_100'] = (
    (master_df['log_raw_risk_cliff'] - min_cliff) / (max_cliff - min_cliff)
) * 100

min_down = master_df['raw_risk_lowincome'].min()
max_down = master_df['raw_risk_lowincome'].max()
master_df['norm_risk_lowincome_100'] = (
    (master_df['raw_risk_lowincome'] - min_down) / (max_down - min_down)
) * 100

# 2. Apply Weights to create the Final Index
master_df['Access_Disruption_Risk_Index'] = (
    (master_df['norm_risk_cliff_100'] * WEIGHT_CLIFF_RISK) +
    (master_df['norm_risk_lowincome_100'] * WEIGHT_LOWINCOME_RISK)
)

master_df['Access_Disruption_Risk_Index_RMS'] = np.sqrt((master_df['norm_risk_cliff_100']**2 + master_df['norm_risk_lowincome_100']**2) / 2)

# ---------------------------------------------------------
# 7. FINAL REVIEW
# ---------------------------------------------------------
print(f"Risk Score calculated using weights: Cliff={WEIGHT_CLIFF_RISK}, Low Income={WEIGHT_LOWINCOME_RISK}")

# Check the top 10 highest risk counties
print("Top 10 High Risk Counties:")
print(master_df[['County FIPS Code', 'norm_risk_cliff_100', 'norm_risk_lowincome_100', 'Access_Disruption_Risk_Index']].sort_values(by='Access_Disruption_Risk_Index', ascending=False).head(10))

#master_df is a df with the Access_Disruption_Risk_Index (which has been normalized and is a value between 0-100) and the data col needed to calculate it
master_df.to_excel('output/master_df_v2.xlsx')

Risk Score calculated using weights: Cliff=0.5, Low Income=0.5
Top 10 High Risk Counties:
     County FIPS Code  norm_risk_cliff_100  norm_risk_lowincome_100  \
2100            55078            62.427400                80.082285   
2053            54095            85.204369                56.515237   
2042            54073            91.224884                48.961199   
2026            54041            80.784354                57.928531   
2014            54017            69.887459                68.328146   
2008            54005            66.196848                71.871270   
2048            54085            81.451840                56.049299   
2049            54087            75.439016                61.992174   
2057            54103            78.974105                58.025802   
2058            54105            77.777680                58.259682   

      Access_Disruption_Risk_Index  
2100                     71.254843  
2053                     70.859803  
2042             